In [10]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

In [ ]:
# Loading the pre-trained Llama3.2-1B model
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B')
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B')
tokenizer.pad_token = tokenizer.eos_token

# Configurar los parámetros de LoRA
lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=32,
    lora_dropout=0.05
)

In [12]:
# Applying the LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [13]:
# Loading dataset for classification
dataset = load_dataset('json', data_files='./dataset/dataset.jsonl')

# Dividir el dataset en entrenamiento y prueba (80% entrenamiento, 20% prueba)
dataset = dataset['train'].train_test_split(test_size=0.2)

In [14]:
# Tokenizar el dataset, concatenando pregunta y respuesta
def encode(examples):
    inputs = [f"{q} {tokenizer.eos_token} {a}" for q, a in zip(examples["input"], examples["output"])]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_datasets = dataset.map(encode, batched=True)

Map: 100%|██████████| 47/47 [00:00<00:00, 4388.13 examples/s]


In [15]:
# Definir data_collator (opcional, dependiendo de tu necesidad)
data_collator = None

In [16]:
# Definir compute_metrics (opcional, dependiendo de tu necesidad)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return {"accuracy": (predictions == labels).astype(float).mean().item()}

In [17]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [18]:
# Crear una instancia de Trainer para el fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [21]:
# Finalmente, entrenar el modelo
trainer.train()

  0%|          | 0/12 [01:18<?, ?it/s]


RuntimeError: MPS backend out of memory (MPS allocated: 20.13 GB, other allocations: 1.13 GB, max allowed: 20.40 GB). Tried to allocate 256 bytes on shared pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# Evaluar el modelo en el conjunto de datos de evaluación
eval_results = trainer.evaluate()

# Imprimir los resultados de la evaluación
print(f"Resultados de la evaluación: {eval_results}")

In [ ]:
# Obtener predicciones
predictions = trainer.predict(tokenized_datasets["test"])

# Convertir las predicciones a etiquetas
predicted_labels = predictions.predictions.argmax(axis=-1)

# Comparar predicciones con etiquetas reales
for i in range(5):  # Mostrar las primeras 5 predicciones
    print(f"Entrada: {tokenizer.decode(tokenized_datasets['test'][i]['input_ids'])}")
    print(f"Predicción: {tokenizer.decode(predicted_labels[i])}")
    print(f"Etiqueta real: {tokenizer.decode(tokenized_datasets['test'][i]['labels'])}")
    print("\n")